![logo](https://github.com/jeanmatheuss/SB-2023-python/blob/main/imgs/logo.jpg?raw=true)

---
## **Projeto 1** | Explorando IA Generativa em Pipeline de ETL com Python
[Jean Matheus](https://www.linkedin.com/in/jean-matheus-sm/)    
[Projeto no GitHub](https://github.com/jeanmatheuss/SB-2023-python)

---


# Personalização de Mensagens de Marketing com IA Generativa

## Descrição
Este projeto foi desenvolvido como parte do Santander Bootcamp 2023 - Ciência de Dados com Python, com o obejtivo de envovler os clientes de maneira mais personalizada usando o poder da Inteligência Artificial Generativa. 

Para isso vamos coletar os dados dos clientes em uma [API](https://sdw-2023-prd.up.railway.app/swagger-ui/index.html) desenvolvida na Santander DevWeek 2023, armazena-los numa lista e a partir dessa lista fazer o carregamento da mensagem personalizada.

## Contexto
Como cientista de dados no Santander, nossa tarefa era melhorar o envolvimento dos clientes por meio de mensagens de marketing altamente personalizadas.    
O objetivo é utilizar  a Inteligência Artificial Generativa para criar mensagens que ressoem com cada cliente individualmente.

Aqui estão as estapas que seguiremos:
1. Extração e Obtenção de Dados dos Clientes
2. Geração de Mensagens Personalizadas
3. Atualização de Dados dos Clientes

### Importando os Pacotes e Bibliotecas

In [1]:
import json
import pandas as pd
import seaborn as sns
import numpy as np
import requests

## 1. Extração dos Dados

In [2]:
# URL da API para a consulta dos clientes

url = 'https://sdw-2023-prd.up.railway.app'

In [3]:
# Criando a lista de clientes diretamente da API

response = requests.get(f'{url}/users')
clients = response.json() if response.status_code == 200 else None


In [35]:
# Vizualizando a estrutura e o primeiro cliente da nossa lista

print(json.dumps(clients[0], indent=2))

{
  "id": 1,
  "name": "Devweekerson",
  "account": {
    "id": 1,
    "number": "01.097954-4",
    "agency": "2030",
    "balance": 624.12,
    "limit": 1000.0
  },
  "card": {
    "id": 1,
    "number": "xxxx xxxx xxxx 1111",
    "limit": 2000.0
  },
  "features": [
    {
      "id": 1,
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pix.svg",
      "description": "PIX"
    },
    {
      "id": 2,
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pay.svg",
      "description": "Pagar"
    },
    {
      "id": 3,
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/transfer.svg",
      "description": "Transferir"
    },
    {
      "id": 4,
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg",
      "description": "Conta Corrente"
    },
    {
      "id": 5,
      "icon": "https://digitalinnovationone.github.io/santander-dev-w

## 3. Geração de Mensagens Personalizadas

In [4]:
# Colocando nossa chave da API do ChatGPT
import os
from getpass import getpass

api_key = getpass("Insira aqui sua API key da OpenIA:")


In [27]:
# Gerando mensagem através da API do ChatGTP

import openai
import time

openai.api_key = api_key

def gerenate_ai_news(user, limit):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Você é um especialista em marketing e em marketing bancário."
            },
            {
                "role": "user",
                "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos em nosso banco, que possuí um limite de: R${limit['limit']} (máximo 140 caracteres)"
            }
        ]
    )
    return completion.choices[0].message.content.strip('\"')


In [28]:
# Escolhendo os 3 últimos clientes do banco de dados da API, gerando a mensagem do ChatGPT através da função 'gerenate_ai_news' 
for user in clients[-3:]:
    limit = user['account']
    news = gerenate_ai_news(user, limit)
    print(news)
    user['news'].append({
        "icon": "https://raw.githubusercontent.com/digitalinnovationone/santander-dev-week-2023-api/665804c8231b32923b03b30ab832ccee33cf2cc3/docs/icons/credit.svg",
        "description": news
    })


Paulo teste1, invista no nosso banco e potencialize seu dinheiro! Alcançar seus objetivos financeiros nunca foi tão fácil. #Investimentos #Banco
Invista seu dinheiro com segurança e rentabilidade no nosso banco. Potencialize seus ganhos com o limite de R$10.000.0.#InvistaConosco
Investimentos no nosso banco: potencialize seu dinheiro com segurança e rentabilidade garantida. Limite de R$10.000,00. Invista já!


## 4. Atualização de Dados dos Clientes

In [29]:
# Fazendo o upload para API com a lista 'news' atualizada, com a nova mensagem gerada pelo ChatGPT, em caso de erro ele mostra o erro

def uptade_user(user):
    response = requests.put(f"{url}/users/{user['id']}", json=user)

    return print(f"User {user['name']} updated: True!") if response.status_code == 200  else print(f"User {user['name']} updated: False! Error: {response.status_code}")

for user in clients[-3:]:
    success = uptade_user(user)


User Paulo teste1 updated: True!
User Paulo teste2 updated: True!
User Paulo teste3 updated: True!
